# Downloading the data

In [1]:
if False:
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials

In [2]:
if False:
    # 1. Authenticate and create the PyDrive client.
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)

In [3]:
if False:
    #2. Get the file
    x_train_downloaded = drive.CreateFile({'id':"1llrLRCwI6gvaTOgGcGl1aigk_97HFW6D"})
    y_train_downloaded = drive.CreateFile({'id':"165kNBNGxqvoSbWoYVhuZNfhJ_MKUxUXB"})
    x_test_downloaded = drive.CreateFile({'id':"1xXEKoy1W1MWUOURUBp_QsZvEpyzwAzIL"})

    x_train_downloaded.GetContentFile('x-train.csv')
    y_train_downloaded.GetContentFile('y-train.csv')
    x_test_downloaded.GetContentFile('x-test.csv')

# Preparing the environment

In [60]:
import pickle

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {tf.keras.__version__}")

TensorFlow version: 2.6.0
Keras version: 2.6.0


In [61]:
# Setup plots

%matplotlib inline
plt.rcParams["figure.figsize"] = 12, 6
%config InlineBackend.figure_format = "retina"
sns.set()

# Data processing

In [69]:
#3. Read file as panda dataframe

x_train = pd.read_csv('x-train.csv')
y_train = pd.read_csv('y-train.csv')
x_test = pd.read_csv('x-test.csv')

In [70]:
test_challengeID = x_test['ChallengeID']

x_train = x_train.drop('ChallengeID', axis=1)
y_train = y_train.drop('ChallengeID', axis=1)
x_test = x_test.drop('ChallengeID', axis=1)

print(x_train.shape)
print(y_train.shape)

(110854, 289)
(110854, 248)


In [71]:
x_train = x_train.drop(x_train.columns[range(248, 289)], axis=1)
x_test = x_test.drop(x_test.columns[range(248, 289)], axis=1)

In [72]:
#x_train.head()
print(x_val.shape)

x_val.head()

(33257, 248)


,acid-jazz,alternative-rock,americana,blaxpoitation,bluegrass,blues-rock,bossa-nova,breakbeat,chill-out,classic-ambient,...,serious,sexy,sinister,sneaky,spectacular,suspenseful,togetherness,uplifting,victorious,warm
40804,0.023602,0.003439,0.000170,0.005550,0.000016,0.000458,0.001247,0.069586,0.151750,0.041836,...,0.045937,0.074555,0.137808,0.012693,0.079792,0.070495,0.001930,0.042277,0.010560,0.029412
104909,0.000056,0.082316,0.000436,0.001022,0.000017,0.015063,0.000020,0.004479,0.000117,0.000020,...,0.012182,0.006554,0.094743,0.002210,0.443428,0.133005,0.000008,0.018382,0.002352,0.002071
107106,0.000008,0.000063,0.006171,0.000035,0.001259,0.000102,0.000321,0.000013,0.001952,0.000317,...,0.044414,0.001405,0.037014,0.000668,0.014133,0.028494,0.001629,0.055537,0.006699,0.193985
102401,0.000191,0.002642,0.000474,0.000411,0.000028,0.000426,0.000048,0.150717,0.035270,0.014367,...,0.188292,0.031874,0.142471,0.003311,0.313070,0.058516,0.001006,0.077642,0.009259,0.161906
76027,0.008058,0.010674,0.000993,0.006206,0.001397,0.000569,0.022605,0.007225,0.038585,0.001003,...,0.004567,0.024457,0.003299,0.001002,0.068770,0.007183,0.000345,0.104573,0.004885,0.111946


# Building the model

The following are Implementations for different models of classifications. Please run only one cell depending on which model you wish to use.

In [33]:
from sklearn.neighbors import KNeighborsClassifier

filename = 'model.sav'

kneighbors = KNeighborsClassifier(n_neighbors=5)
kneighbors.fit(x_train, y_train)
pickle.dump(kneighbors, open(filename, 'wb'))

In [ ]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.svm import SVC

binay_relevance = BinaryRelevance(classifier = SVC(probability=True))

binay_relevance.fit(x_train, y_train)

In [ ]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier

n_classes = 248
forest = RandomForestClassifier(random_state=1)
multi_target_forest = MultiOutputClassifier(forest, n_jobs=-1)
multi_target_forest.fit(x_train, y_train)

# Evaluating validation data

In [48]:
from sklearn.metrics import f1_score

#y_val_predicted = kneighbors.predict(x_val)
#y_val_predicted = binay_relevance.predict(x_val)
#y_val_predicted = classifier.predict(x_val.iloc[[0]])
y_val_predicted = multi_target_forest.predict(x_val)


score = f1_score(y_val, y_val_predicted, average='weighted')

IndexError: list index out of range

In [ ]:
print(score)

# Predicting and exporting test results 

In [59]:
y_test = classifier.predict(x_test)

IndexError: list index out of range

In [17]:
trained_dataframe = pd.DataFrame(y_test, columns=y_train.columns.values)
trained_dataframe.insert(0, 'ChallengeID', test_challengeID)
trained_dataframe.to_csv("result.csv", index=False)
trained_dataframe

,ChallengeID,acid-jazz,alternative-rock,americana,blaxpoitation,bluegrass,blues-rock,bossa-nova,breakbeat,chill-out,...,serious,sexy,sinister,sneaky,spectacular,suspenseful,togetherness,uplifting,victorious,warm
0,110854,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,110855,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,110856,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,110857,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,110858,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47610,158464,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
47611,158465,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
47612,158466,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
47613,158467,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
